In [1]:
import numpy as np
import pandas as pd
import ultralytics
import supervision
import torch


# Tracking and Counting with Line

In [3]:
import cv2
from collections import defaultdict
from ultralytics import YOLO
model = YOLO('yolov8s.pt')
cap = cv2.VideoCapture("testing/Bottle.mp4")
START = (1432, -2)
END = (1432, 1300)

# Function to determine if a point is above or below the line
def is_above_line(point, start, end):
    return (point[0] - start[0]) * (end[1] - start[1]) - (point[1] - start[1]) * (end[0] - start[0]) > 0

track_history = defaultdict(list)

# Create a dictionary to keep track of objects that have crossed the line
crossed_objects = {}
cross_count = 0

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('counter.mp4', fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 tracking on the frame, persisting tracks between frames
    results = model.track(frame, classes=[39], persist=True, tracker="bytetrack.yaml")

    # Get the boxes and track IDs
    boxes = results[0].boxes.xywh.cpu().numpy()
    track_ids = results[0].boxes.id.cpu().numpy()

    # Visualize the results on the frame
    annotated_frame = frame.copy() 

    # Plot the tracks and count objects crossing the line
    for box, track_id in zip(boxes, track_ids):
        x, y, w, h = box
        center_x, center_y = int(x), int(y)
        track = track_history[track_id]
        track.append((center_x, center_y))

        # Initialize the object position if it's not in the dictionary
        if track_id not in crossed_objects:
            crossed_objects[track_id] = "above" if is_above_line((center_x, center_y), START, END) else "below"

        if crossed_objects[track_id] == "above" and not is_above_line((center_x, center_y), START, END):
            cross_count += 1
            crossed_objects[track_id] = "below"
        elif crossed_objects[track_id] == "below" and is_above_line((center_x, center_y), START, END):
            cross_count += 1
            crossed_objects[track_id] = "above"

        # Get the text size for the class name
        text_size, _ = cv2.getTextSize('bottle', cv2.FONT_HERSHEY_SIMPLEX, 1, 2)

        # Draw a red filled rectangle around the class name
        text_x = center_x - text_size[0] // 2
        text_y = center_y + text_size[1] // 2
        cv2.rectangle(annotated_frame, (text_x - 5, text_y - text_size[1] - 5),
                      (text_x + text_size[0] + 5, text_y + 5), (0, 0, 255), -1)

        # Draw the class name in white
        cv2.putText(annotated_frame, 'bottle', (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.line(annotated_frame, START, END, (0, 255, 255), 4) 

    # Write count of objects on each frame
    count_text = f"Count: {cross_count}"
    count_text_size, _ = cv2.getTextSize(count_text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    # Calculate the position for the count text
    count_text_x = annotated_frame.shape[1] - count_text_size[0] - 10 
    count_text_y = 30
    # Draw the blue filled rectangle around the count text
    cv2.rectangle(annotated_frame, (count_text_x - 5, count_text_y - count_text_size[1] - 5),
                  (count_text_x + count_text_size[0] + 5, count_text_y + 5), (255, 0, 0), -1)
    # Write the count text in white
    cv2.putText(annotated_frame, count_text, (count_text_x, count_text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # frame with annotations
    out.write(annotated_frame)
cap.release()
out.release()



0: 384x640 4 bottles, 187.5ms
Speed: 0.0ms preprocess, 187.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 bottles, 171.9ms
Speed: 0.0ms preprocess, 171.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 bottles, 187.5ms
Speed: 15.6ms preprocess, 187.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 bottles, 156.3ms
Speed: 15.6ms preprocess, 156.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 bottles, 151.9ms
Speed: 0.0ms preprocess, 151.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 bottles, 160.4ms
Speed: 7.9ms preprocess, 160.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 bottles, 174.1ms
Speed: 0.0ms preprocess, 174.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 bottles, 151.5ms
Speed: 4.0ms preprocess, 151.5ms inference, 13.5ms postprocess p